### Python notebook for Capstone project
#### - Call Foursquare API to get a list of coffee shops and their ratings near Rockefeller Center, NY
#### - Preprocess and analyse the data to find out the top 5 coffee shops 
#### - Present the results in a map

In [1]:
#import libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

In [2]:
#set Foursquare client ID, secret and token

CLIENT_ID = 'MBIYWB1GPHLEJ4JQGYUJNMVLJN1NKBKNZ30LBP40GHIELBTR' # your Foursquare ID
CLIENT_SECRET = 'HM5ES3FCQPOWKTBD22VJP3N4CXVNRW3FGIVZHIHVHBG1H2OA' # your Foursquare Secret
ACCESS_TOKEN = 'OEKZ4J0WPJEYBMR2Z4LRYWEKLMO5DCHSDS4UYDHAAV1XHL2F' # your FourSquare Access Token
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MBIYWB1GPHLEJ4JQGYUJNMVLJN1NKBKNZ30LBP40GHIELBTR
CLIENT_SECRET:HM5ES3FCQPOWKTBD22VJP3N4CXVNRW3FGIVZHIHVHBG1H2OA


In [3]:
#set the location of Rockefeller Center, New York

address = 'Rockefeller Center, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.75925755 -73.97995686116377


In [4]:
#set query to search for coffee shops within 1km and limits the number of results to 100

search_query = 'Coffee'
radius = 1000
LIMIT = 50

In [5]:
#URI to search coffee shops in Foursquare

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MBIYWB1GPHLEJ4JQGYUJNMVLJN1NKBKNZ30LBP40GHIELBTR&client_secret=HM5ES3FCQPOWKTBD22VJP3N4CXVNRW3FGIVZHIHVHBG1H2OA&ll=40.75925755,-73.97995686116377&oauth_token=OEKZ4J0WPJEYBMR2Z4LRYWEKLMO5DCHSDS4UYDHAAV1XHL2F&v=20180604&query=Coffee&radius=1000&limit=50'

In [6]:
#call Foursquare API to get the results of coffees shops

results = requests.get(url).json()

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = pd.json_normalize(venues)
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,59ce5ad9a2a6ce39476dbf1d,Blue Bottle Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1611707046,False,10 E 53rd St,5th Ave,40.759465,-73.975672,"[{'label': 'display', 'lat': 40.75946472799885...",...,NY,United States,"[10 E 53rd St (5th Ave), New York, NY 10022, U...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,581b3b5b5c3b5845451935e0,Gregorys Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1611707046,False,762 7th Ave,West 50th Street,40.761250,-73.983359,"[{'label': 'display', 'lat': 40.76125, 'lng': ...",...,NY,United States,"[762 7th Ave (West 50th Street), New York, NY ...",1097646,https://www.seamless.com/menu/gregorys-coffee-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,437216324
2,5a10570ebb2a4740244092d3,Remi Flowers & Coffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611707046,False,906 2nd Ave,btwn E 48th & E 49th St,40.753751,-73.969075,"[{'label': 'display', 'lat': 40.7537510182011,...",...,NY,United States,"[906 2nd Ave (btwn E 48th & E 49th St), New Yo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4e6fe48c1fc7d3fc71285ec2,Gregorys Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1611707046,False,12 E 46th St,btwn Madison & 5th Ave,40.755434,-73.978271,"[{'label': 'display', 'lat': 40.7554338, 'lng'...",...,NY,United States,"[12 E 46th St (btwn Madison & 5th Ave), New Yo...",1043308,https://www.seamless.com/menu/gregorys-coffee-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
4,52f8ed59498ede6f715776d8,Sant Ambroeus Coffee Bar,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1611707046,False,540 Park Ave,at 61st St,40.764365,-73.969354,"[{'label': 'display', 'lat': 40.76436530280706...",...,NY,United States,"[540 Park Ave (at 61st St), New York, NY 10065...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['id','name', 'categories'] + [col for col in df.columns if col.startswith('location.')]
df_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

# add new column rating
df_filtered['rating'] = 0

df_filtered.head()

,id,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,rating
0,59ce5ad9a2a6ce39476dbf1d,Blue Bottle Coffee,Coffee Shop,10 E 53rd St,5th Ave,40.759465,-73.975672,"[{'label': 'display', 'lat': 40.75946472799885...",362,10022,US,Midtown East,New York,NY,United States,"[10 E 53rd St (5th Ave), New York, NY 10022, U...",0
1,581b3b5b5c3b5845451935e0,Gregorys Coffee,Coffee Shop,762 7th Ave,West 50th Street,40.761250,-73.983359,"[{'label': 'display', 'lat': 40.76125, 'lng': ...",362,10019,US,NaN,New York,NY,United States,"[762 7th Ave (West 50th Street), New York, NY ...",0
2,5a10570ebb2a4740244092d3,Remi Flowers & Coffee,Café,906 2nd Ave,btwn E 48th & E 49th St,40.753751,-73.969075,"[{'label': 'display', 'lat': 40.7537510182011,...",1103,10017,US,"Turtle Bay, New York, NY",New York,NY,United States,"[906 2nd Ave (btwn E 48th & E 49th St), New Yo...",0
3,4e6fe48c1fc7d3fc71285ec2,Gregorys Coffee,Coffee Shop,12 E 46th St,btwn Madison & 5th Ave,40.755434,-73.978271,"[{'label': 'display', 'lat': 40.7554338, 'lng'...",448,10017,US,NaN,New York,NY,United States,"[12 E 46th St (btwn Madison & 5th Ave), New Yo...",0
4,52f8ed59498ede6f715776d8,Sant Ambroeus Coffee Bar,Café,540 Park Ave,at 61st St,40.764365,-73.969354,"[{'label': 'display', 'lat': 40.76436530280706...",1059,10065,US,NaN,New York,NY,United States,"[540 Park Ave (at 61st St), New York, NY 10065...",0


In [14]:
#For each coffee shop, call Foursquare API to get the coffee shop rating
for idx in df_filtered.index:
    venue_id = df_filtered["id"][idx]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        df_filtered.loc[idx,'rating'] = result['response']['venue']['rating']
    except:
        df_filtered.loc[idx,'rating'] = 0

In [34]:
#save the dataset to csv file to avoid having to use Foursquare premium call in the future
df_filtered.to_csv("capstone-data.csv")

#Briefly analyze the data
df_filtered.describe()

,lat,lng,distance,rating
count,50.000000,50.000000,50.000000,50.000000
mean,40.757958,-73.980008,601.820000,3.246000
std,0.003844,0.006058,308.512011,3.889085
min,40.751492,-73.992898,64.000000,0.000000
25%,40.755270,-73.983542,345.500000,0.000000
50%,40.757765,-73.981687,596.000000,0.000000
75%,40.759999,-73.975786,876.000000,7.575000
max,40.767770,-73.966936,1103.000000,9.100000


In [35]:
#sort by rating with descending order and get the first 5 rows
df_best = df_filtered.sort_values('rating',ascending=False)[0:5]
df_best

,id,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,rating
37,54566365498e4fb5ed4fe987,Blue Bottle Coffee,Coffee Shop,54 W 40th St,at 6th Ave,40.753027,-73.984140,"[{'label': 'display', 'lat': 40.753027, 'lng':...",778,10018,US,NaN,New York,NY,United States,"[54 W 40th St (at 6th Ave), New York, NY 10018...",9.1
28,4f5e82dae4b01989c438cbb6,Blue Bottle Coffee,American Restaurant,1 Rockefeller Plaza Suite N,at W 49th St,40.758217,-73.978714,"[{'label': 'display', 'lat': 40.758217, 'lng':...",156,10020,US,NaN,New York,NY,United States,"[1 Rockefeller Plaza Suite N (at W 49th St), N...",8.9
2,5a10570ebb2a4740244092d3,Remi Flowers & Coffee,Café,906 2nd Ave,btwn E 48th & E 49th St,40.753751,-73.969075,"[{'label': 'display', 'lat': 40.7537510182011,...",1103,10017,US,"Turtle Bay, New York, NY",New York,NY,United States,"[906 2nd Ave (btwn E 48th & E 49th St), New Yo...",8.9
0,59ce5ad9a2a6ce39476dbf1d,Blue Bottle Coffee,Coffee Shop,10 E 53rd St,5th Ave,40.759465,-73.975672,"[{'label': 'display', 'lat': 40.75946472799885...",362,10022,US,Midtown East,New York,NY,United States,"[10 E 53rd St (5th Ave), New York, NY 10022, U...",8.6
4,52f8ed59498ede6f715776d8,Sant Ambroeus Coffee Bar,Café,540 Park Ave,at 61st St,40.764365,-73.969354,"[{'label': 'display', 'lat': 40.76436530280706...",1059,10065,US,NaN,New York,NY,United States,"[540 Park Ave (at 61st St), New York, NY 10065...",8.4


In [36]:
#display the top 5 coffee shops with the best rating in table format
df_best[["name","address","postalCode","rating"]]

,name,address,postalCode,rating
37,Blue Bottle Coffee,54 W 40th St,10018,9.1
28,Blue Bottle Coffee,1 Rockefeller Plaza Suite N,10020,8.9
2,Remi Flowers & Coffee,906 2nd Ave,10017,8.9
0,Blue Bottle Coffee,10 E 53rd St,10022,8.6
4,Sant Ambroeus Coffee Bar,540 Park Ave,10065,8.4


In [37]:
#Show the coffee shops in a map
map_coffee = folium.Map(location=[latitude, longitude], zoom_start=15)
label = folium.Popup(address, parse_html=True)
folium.Marker([latitude, longitude],icon=folium.Icon(color="black", prefix="fa", icon="building"),\
              popup=label).add_to(map_coffee)

for lat, lon, name, rating in zip(df_best['lat'], df_best['lng'], df_best['name'], df_best['rating']):
    label = folium.Popup(str(name) + ". " + "Rating: " + str(rating), parse_html=True)
    folium.Marker([lat, lon],icon=folium.Icon(color="blue",prefix="fa",icon="coffee"),\
                        popup=label).add_to(map_coffee)
    

map_coffee